# Convert mw NetCDF to Tif

**Purpose of script:**

Convert the microwave data from NetCDF format to Tif format. 

- In: mw NetCDF data (one file)
- Out: mw data in Tif foramt (multiple files)

*Note: Dates are specified by defining the start and end date of desired period in the cell below.*

In [1]:
import numpy as np
import xarray
import rasterio
import pandas as pd

Inputs and relevant paths

In [2]:
# start_date = "2019-06-01"
# end_date = "2019-06-30"

mw_path = r"../Data/microwave-rs/CumJour-North-ssmi-1979-2022-H19.nc"
#mw_out_path = r'../Data/microwave-rs/mw_tif/'

# TEMP for testing non-melt areas:
mw_out_path = r'../Data/microwave-rs/mw_non_melt_testing_tif/'

## Data Preparation

Data load

In [5]:
xarray.open_dataset('testData.nc')

ValueError: did not find a match in any of xarray's currently installed IO backends ['netcdf4', 'h5netcdf', 'scipy', 'cfgrib', 'pydap', 'rasterio', 'zarr']. Consider explicitly selecting one of the installed engines via the ``engine`` parameter, or installing additional IO dependencies, see:
https://docs.xarray.dev/en/stable/getting-started-guide/installing.html
https://docs.xarray.dev/en/stable/user-guide/io.html

In [3]:
dataset_mw = xarray.open_dataset('testData.nc').rio.write_crs(
        "epsg:3413", inplace=True)

/mnt/c/Users/linas/OneDrive/Documents/ITU/Semester3/RP_Greenland/RP_Greenland/.venv/lib/python3.9/site-packages/xarray/backends/plugins.py:71: RuntimeWarning: Engine 'cfgrib' loading failed:
Cannot find the ecCodes library
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)


ValueError: did not find a match in any of xarray's currently installed IO backends ['netcdf4', 'h5netcdf', 'scipy', 'cfgrib', 'pydap', 'rasterio', 'zarr']. Consider explicitly selecting one of the installed engines via the ``engine`` parameter, or installing additional IO dependencies, see:
https://docs.xarray.dev/en/stable/getting-started-guide/installing.html
https://docs.xarray.dev/en/stable/user-guide/io.html

Date range

In [5]:
#period = [start_date, end_date]
#datelist = pd.date_range(*period)

# TEMP for testing non-melt areas:
period = [['2017-06-01', '2017-08-31'],
          ['2018-06-01', '2018-08-31'],
          ['2019-06-01', '2019-08-31'], 
          ['2020-06-01', '2020-08-31'],
          ['2021-06-01', '2021-08-31'],
          ['2022-06-01', '2022-08-31']]

for i in period:
    datelist_ = pd.date_range(*i)
    datelist = datelist + datelist_

## Convert to Tif

Main

In [5]:
for i in datelist:
    #print(i)
    dataset_day = dataset_mw.where(dataset_mw["time"] == i, drop=True)
    data_mw_day = dataset_day["melt"]
    out_path_and_name = f"{mw_out_path}{i.date()}_mw.tif"
    data_mw_day.rio.to_raster(out_path_and_name)

    with rasterio.open(out_path_and_name) as src_mw:
        mw = src_mw.read()
        np.nan_to_num(mw, nan=-1, copy= False) # inplace
        kwargs1 = src_mw.meta.copy()

    # overwrite mw with new band (fixed nan)
    with rasterio.open(out_path_and_name, 'w', **kwargs1) as dst:
        dst.write_band(1, mw[0])

# would be nice to do that with just one package